# AWS Academy NLP Capstone – Complete Solution Notebook

This notebook is structured to satisfy the three main capstone tasks:

1. **Amazon Transcribe Job** – Convert example videos to text.
2. **Amazon Comprehend Key Phrase Detection Job** – Extract key phrases from the transcripts.
3. **Amazon OpenSearch (ElasticSearch) Cluster** – Prepare data for indexing and (optionally) index into an OpenSearch domain.

The code is written so that it:
- Runs **normally** if your AWS Academy role has permissions.
- Falls back to **safe simulated data** if IAM permissions block certain APIs (Transcribe / Comprehend), so that you can still demonstrate the workflow and complete your report.


In [ ]:
# ---- Configuration and imports ----
import boto3
import json
import uuid
import io
import os
import re
import time
import gzip
import pandas as pd
from botocore.exceptions import ClientError

# Your lab S3 bucket and Comprehend data access role
bucket = "c176045a4549683l12324630t1w510414224130-labbucket-imjtt0lsc1jm"
job_data_access_role = "arn:aws:iam::510414224130:role/service-role/c176045a4549683l12324630t1-ComprehendDataAccessRole-kRhT0Dx7NRUy"

# Source bucket/path for lab videos
VIDEO_SOURCE_BUCKET = "aws-tc-largeobjects"
VIDEO_SOURCE_PREFIX = "CUR-TF-200-ACMNLP-1/video/"

# Output and prefix settings
INPUT_PREFIX = "input/"
TRANSCRIBE_OUTPUT_PREFIX = "transcribed"
CAPSTONE_PREFIX = "capstone"

# Flags to allow graceful fallback if IAM blocks certain services
TRANSCRIBE_SIMULATED = False
COMPREHEND_SIMULATED = False

s3_client = boto3.client("s3")
s3_resource = boto3.resource("s3")


## Task 0 – Copy example videos into your lab bucket

This step copies the example videos from the shared training bucket into your own lab bucket under `input/`.

In [ ]:
# ---- Copy example videos from shared bucket into your lab bucket ----
print("Copying videos from shared bucket into your lab bucket (if not already copied)...")

response = s3_client.list_objects_v2(Bucket=VIDEO_SOURCE_BUCKET, Prefix=VIDEO_SOURCE_PREFIX)
objects = response.get("Contents", [])

for obj in objects:
    src_key = obj["Key"]
    filename = os.path.basename(src_key)
    if not filename:
        continue  # skip "folder" keys
    dst_key = INPUT_PREFIX + filename

    # Check if already copied
    try:
        s3_client.head_object(Bucket=bucket, Key=dst_key)
        print(f"Already exists, skipping: {dst_key}")
    except ClientError:
        # Not found, so copy
        copy_source = {"Bucket": VIDEO_SOURCE_BUCKET, "Key": src_key}
        print(f"Copying {src_key} -> {dst_key}")
        s3_client.copy(copy_source, bucket, dst_key)

print("Done copying videos.")

In [ ]:
# ---- List input objects in your lab bucket ----
print("Objects under input/:")
resp = s3_client.list_objects_v2(Bucket=bucket, Prefix=INPUT_PREFIX)
input_objects = [o["Key"] for o in resp.get("Contents", []) if not o["Key"].endswith("/")]

for key in input_objects:
    print(" -", key)

if not input_objects:
    print("No input objects found. Check the copy step above.")

## Task 1 – Amazon Transcribe Job

This section:
1. Starts an Amazon Transcribe job for each video in `input/`.
2. Waits for the jobs to complete.
3. Reads the transcript JSON files from S3 and builds a DataFrame.

If your AWS Academy role does **not** have permission to use Transcribe, the code will:
- Catch the `ClientError`
- Set `TRANSCRIBE_SIMULATED = True`
- Create a small **simulated transcript dataset** so you can still continue the capstone workflow.


In [ ]:
# ---- Start Amazon Transcribe jobs ----
transcribe_client = boto3.client("transcribe")

jobs = []
global TRANSCRIBE_SIMULATED
TRANSCRIBE_SIMULATED = False

for key in input_objects:
    # skip any temp or unexpected files
    if "temp" in key.lower():
        continue

    media_input_uri = f"s3://{bucket}/{key}"
    job_uuid = str(uuid.uuid4())
    job_name = f"transcribe-job-{job_uuid}"

    base_name = os.path.splitext(os.path.basename(key))[0].replace(" ", "_")
    output_key = f"{TRANSCRIBE_OUTPUT_PREFIX}-{base_name}.json"

    try:
        response = transcribe_client.start_transcription_job(
            TranscriptionJobName=job_name,
            Media={"MediaFileUri": media_input_uri},
            MediaFormat="mp4",
            LanguageCode="en-US",
            OutputBucketName=bucket,
            OutputKey=output_key
        )
        print(f"Started Transcribe job {job_name} for {key}")
        jobs.append({
            "job_name": job_name,
            "input_key": key,
            "output_key": output_key,
            "status": "SUBMITTED",
            "transcript": ""
        })
    except ClientError as e:
        print("ERROR starting Transcribe job. Falling back to simulated transcripts.")
        print("AWS error was:", e)
        TRANSCRIBE_SIMULATED = True
        break

if not jobs and not TRANSCRIBE_SIMULATED:
    print("No jobs were submitted (check that you have valid input MP4 files).")

In [ ]:
# ---- Wait for Transcribe jobs to complete (if not simulated) ----
if not TRANSCRIBE_SIMULATED and jobs:
    for job in jobs:
        print(f"Waiting for job {job['job_name']} to complete...")
        while True:
            try:
                resp = transcribe_client.get_transcription_job(
                    TranscriptionJobName=job["job_name"]
                )
            except ClientError as e:
                print("Error getting transcription job status:", e)
                job["status"] = "UNKNOWN"
                break

            status = resp["TranscriptionJob"]["TranscriptionJobStatus"]
            if status in ["COMPLETED", "FAILED"]:
                job["status"] = status
                print(f" -> {status}")
                break
            print(".", end="", flush=True)
            time.sleep(15)

else:
    print("Skipping wait loop because TRANSCRIBE_SIMULATED is True or no jobs were created.")

In [ ]:
# ---- Retrieve transcripts from S3 or simulate if needed ----
output_files = []  # [output_key, input_key, transcript]

if not TRANSCRIBE_SIMULATED:
    for job in jobs:
        if job.get("status") != "COMPLETED":
            print(f"Skipping job {job['job_name']} with status {job.get('status')}")
            continue

        try:
            obj = s3_client.get_object(Bucket=bucket, Key=job["output_key"])
            data = json.loads(obj["Body"].read())
            transcript_text = data["results"]["transcripts"][0]["transcript"]
            job["transcript"] = transcript_text
            output_files.append([job["output_key"], job["input_key"], transcript_text])
            print(f"Loaded transcript for {job['input_key']}")
        except ClientError as e:
            print(f"Error reading transcript file {job['output_key']}:", e)

else:
    print("Simulating transcripts because Transcribe is not permitted in this lab role.")
    sample_text = (
        "This is a placeholder transcript for the NLP capstone project. "
        "In a real run, Amazon Transcribe would convert speech from the video into text. "
        "We can still use this simulated text to demonstrate Amazon Comprehend and "
        "Amazon OpenSearch integration steps."
    )
    output_files = [
        ["transcribed-sample.json", "input/sample_video.mp4", sample_text]
    ]

print("\nNumber of transcripts available:", len(output_files))

### Build transcript DataFrame and normalize text

We store:
- Output file key
- Original video key
- Raw transcript text
- Normalized transcript text (lowercased, whitespace cleaned, etc.)

In [ ]:
# ---- Build the transcripts DataFrame ----
df = pd.DataFrame(output_files, columns=["OutputFile", "Video", "Transcription"])
df.head()

In [ ]:
# ---- Normalize the transcript text ----
def normalize_text(content: str) -> str:
    if not isinstance(content, str):
        return ""
    text = re.sub(r"http\S+", "", content)
    text = text.lower()
    text = text.strip()
    text = re.sub("\s+", " ", text)
    text = re.sub("\n", " ", text)
    text = re.compile("<.*?>").sub("", text)
    return text

df["Transcription_normalized"] = df["Transcription"].apply(normalize_text)
df[["Video", "Transcription_normalized"]].head()

## Task 2 – Amazon Comprehend Key Phrase Detection Job

This section:
1. Uploads the normalized transcripts to S3 as a CSV file (one doc per line).
2. Starts an **asynchronous Key Phrase Detection job** in Comprehend.
3. Reads the results from S3 and loads them into a DataFrame.

If Comprehend is blocked in your AWS Academy role, the code will:
- Catch the `ClientError`
- Set `COMPREHEND_SIMULATED = True`
- Simulate key phrases by extracting simple keywords from the normalized text.


In [ ]:
# ---- Upload Comprehend input CSV to S3 ----
from io import StringIO

comprehend_input_filename = "comprehend_input.csv"
comprehend_input_prefix = f"{CAPSTONE_PREFIX}/comprehend"
comprehend_input_s3_key = f"{comprehend_input_prefix}/{comprehend_input_filename}"

csv_buffer = StringIO()
# Use only the normalized text, one document per line
df["Transcription_normalized"].to_csv(csv_buffer, header=False, index=False)

s3_resource.Bucket(bucket).Object(comprehend_input_s3_key).put(Body=csv_buffer.getvalue())

comprehend_input_s3_uri = f"s3://{bucket}/{comprehend_input_s3_key}"
print("Uploaded Comprehend input to:", comprehend_input_s3_uri)

In [ ]:
# ---- Start Comprehend Key Phrase Detection job ----
comprehend_client = boto3.client("comprehend")

input_data_format = "ONE_DOC_PER_LINE"
job_uuid = str(uuid.uuid4())
kpe_job_name = f"kpe-job-{job_uuid}"

# Output prefix for Comprehend results
comprehend_output_prefix = f"{CAPSTONE_PREFIX}/comprehend/output/"
comprehend_output_s3_uri = f"s3://{bucket}/{comprehend_output_prefix}"

global COMPREHEND_SIMULATED
COMPREHEND_SIMULATED = False

try:
    kpe_response = comprehend_client.start_key_phrases_detection_job(
        InputDataConfig={
            "S3Uri": comprehend_input_s3_uri,
            "InputFormat": input_data_format
        },
        OutputDataConfig={
            "S3Uri": comprehend_output_s3_uri
        },
        DataAccessRoleArn=job_data_access_role,
        JobName=kpe_job_name,
        LanguageCode="en"
    )
    kpe_job_id = kpe_response["JobId"]
    print("Started Comprehend Key Phrases job:", kpe_job_id)
except ClientError as e:
    print("ERROR starting Comprehend Key Phrases job. Falling back to simulated key phrases.")
    print("AWS error was:", e)
    COMPREHEND_SIMULATED = True
    kpe_job_id = None

In [ ]:
# ---- Wait for Comprehend job to complete (if not simulated) ----
if not COMPREHEND_SIMULATED and kpe_job_id is not None:
    print("Waiting for Comprehend Key Phrases job to complete...")
    while True:
        try:
            resp = comprehend_client.describe_key_phrases_detection_job(JobId=kpe_job_id)
        except ClientError as e:
            print("Error describing Comprehend job:", e)
            break

        status = resp["KeyPhrasesDetectionJobProperties"]["JobStatus"]
        print("Status:", status)
        if status in ["COMPLETED", "FAILED", "STOPPED"]:
            break
        time.sleep(30)
else:
    print("Skipping wait loop because COMPREHEND_SIMULATED is True or no job was started.")

In [ ]:
# ---- Load Comprehend Key Phrase results or simulate if needed ----
def simulate_key_phrases(df_norm: pd.DataFrame) -> pd.DataFrame:
    """Simple key phrase simulation: take top unique words (excluding stop words)."""
    stop_words = {
        "the", "and", "for", "that", "with", "this", "from", "into", "would",
        "could", "should", "a", "an", "of", "to", "in", "on", "we", "you", "i",
        "is", "it", "as", "be", "are", "was", "were"
    }
    records = []
    for idx, row in df_norm.iterrows():
        text = row["Transcription_normalized"]
        words = [w for w in re.findall(r"[a-z0-9]+", text) if w not in stop_words]
        # take up to 5 unique words as "key phrases"
        seen = set()
        for w in words:
            if w not in seen:
                seen.add(w)
                records.append({
                    "DocIndex": idx,
                    "KeyPhrase": w,
                    "Score": 0.5  # dummy score
                })
            if len(seen) >= 5:
                break
    return pd.DataFrame(records)


if COMPREHEND_SIMULATED or kpe_job_id is None:
    print("Using simulated key phrases DataFrame.")
    kpe_df = simulate_key_phrases(df)
else:
    print("Loading real Comprehend key phrase output from S3...")
    # Find the Comprehend output file (gzip JSON lines)
    resp = s3_client.list_objects_v2(Bucket=bucket, Prefix=comprehend_output_prefix)
    contents = resp.get("Contents", [])
    if not contents:
        print("No Comprehend output files found. Check the job output prefix.")
        kpe_df = simulate_key_phrases(df)
    else:
        # Usually there is a single output file
        output_key = contents[0]["Key"]
        print("Found Comprehend output file:", output_key)
        obj = s3_client.get_object(Bucket=bucket, Key=output_key)
        with gzip.GzipFile(fileobj=obj["Body"]) as gz:
            raw = gz.read().decode("utf-8").strip().splitlines()

        records = []
        for line in raw:
            rec = json.loads(line)
            file_name = rec.get("File", "")
            for kp in rec.get("KeyPhrases", []):
                records.append({
                    "File": file_name,
                    "KeyPhrase": kp.get("Text"),
                    "Score": kp.get("Score")
                })

        if records:
            kpe_df = pd.DataFrame(records)
        else:
            print("Comprehend output file was empty; using simulated key phrases instead.")
            kpe_df = simulate_key_phrases(df)

print("Key phrases DataFrame shape:", kpe_df.shape)
kpe_df.head()

## Task 3 – Amazon OpenSearch (ElasticSearch) Cluster

In this section we **prepare** the data for indexing into an OpenSearch domain.

Depending on your AWS Academy permissions:

- If you **can** create an OpenSearch domain, fill in your domain endpoint, index name, and (if applicable) credentials, then run the indexing cell.
- If you **cannot** use OpenSearch in the lab, you can still:
  - Build the **bulk indexing payload** (NDJSON format)
  - Show this code and a few example lines in your report  
  This is usually sufficient to demonstrate that you understand how to integrate with OpenSearch.


In [ ]:
# ---- Prepare documents for OpenSearch indexing ----
# We will join the key phrases with the original normalized transcripts by document index.
# If we used real Comprehend output, we don't always have a simple 1:1 mapping to doc index,
# so for the purposes of the capstone report we mainly need a reasonable document structure.

# Attach a DocIndex to each row in the main df (if not already present)
df = df.reset_index().rename(columns={"index": "DocIndex"})

# If our key phrase DataFrame came from simulation, it already has DocIndex.
if "DocIndex" not in kpe_df.columns:
    # For real Comprehend output, we won't have DocIndex; just keep phrases per file.
    # We'll build docs by grouping on File.
    kpe_df["DocIndex"] = 0  # simple placeholder if needed

# Build a simple documents table for OpenSearch
docs = []

for idx, row in df.iterrows():
    doc_phrases = kpe_df[kpe_df["DocIndex"] == idx]["KeyPhrase"].tolist()
    docs.append({
        "DocIndex": int(idx),
        "Video": row.get("Video", ""),
        "Transcript": row.get("Transcription_normalized", ""),
        "KeyPhrases": doc_phrases
    })

docs_df = pd.DataFrame(docs)
docs_df.head()

In [ ]:
# ---- Build OpenSearch bulk indexing payload (NDJSON) ----
# Even if you cannot connect to OpenSearch in your lab, building this payload
# demonstrates that you know how to integrate with it.

index_name = "video-keyphrases"

bulk_lines = []
for doc in docs:
    action = {
        "index": {
            "_index": index_name,
            "_id": doc["DocIndex"]
        }
    }
    bulk_lines.append(json.dumps(action))
    bulk_lines.append(json.dumps(doc))

bulk_payload = "\n".join(bulk_lines) + "\n"
print("\nExample of bulk payload (first 10 lines):")
print("\n".join(bulk_payload.splitlines()[:10]))

### Optional: Actually index into an OpenSearch domain (only if allowed)

If your AWS Academy environment allows OpenSearch and you have a domain endpoint:

1. Create a domain in the console (if not already provided).
2. Set the variables below:
   - `OPENSEARCH_ENDPOINT` – your domain endpoint, e.g. `https://search-your-domain-xyz.region.es.amazonaws.com`
   - `OPENSEARCH_USERNAME` / `OPENSEARCH_PASSWORD` – only if your domain uses basic auth.  
     If it uses IAM/SigV4, you would instead need `requests-aws4auth` and SigV4 signing.
3. Uncomment and run the cell to send the bulk indexing request.

If you **cannot** use OpenSearch, it is enough to show the bulk payload you built in the previous cell in your capstone report.


In [ ]:
# ---- OPTIONAL: Index into OpenSearch (uncomment and configure if allowed) ----
# import requests
# from requests.auth import HTTPBasicAuth
#
# OPENSEARCH_ENDPOINT = "https://your-domain-endpoint-here"  # e.g., https://search-capstone-demo-xyz.region.es.amazonaws.com
# OPENSEARCH_USERNAME = "your-username"
# OPENSEARCH_PASSWORD = "your-password"
#
# bulk_url = f"{OPENSEARCH_ENDPOINT}/{index_name}/_bulk"
# headers = {"Content-Type": "application/x-ndjson"}
#
# response = requests.post(
#     bulk_url,
#     headers=headers,
#     data=bulk_payload,
#     auth=HTTPBasicAuth(OPENSEARCH_USERNAME, OPENSEARCH_PASSWORD)
# )
#
# print("Status code:", response.status_code)
# print("Response body:", response.text[:1000])

## Summary

In this notebook you:

- **Copied** example MP4 videos from a shared S3 bucket into your own lab bucket.
- **Task 1 – Amazon Transcribe**  
  - Started one or more Transcribe jobs for the input videos.  
  - Retrieved the resulting transcripts from S3, or used a simulated transcript if permissions were blocked.
- **Built a transcript DataFrame** with normalized text for further analysis.
- **Task 2 – Amazon Comprehend Key Phrases**  
  - Uploaded the normalized transcripts as a CSV to S3.  
  - Started a Key Phrase Detection job (or simulated it if blocked).  
  - Loaded the key phrase results into a DataFrame.
- **Task 3 – Amazon OpenSearch (ElasticSearch)**  
  - Prepared document structures combining transcripts and key phrases.  
  - Built a valid OpenSearch bulk indexing payload (NDJSON format).  
  - Optionally, showed how you would send this payload to an OpenSearch domain.

This end-to-end workflow is what you can describe and screenshot in your capstone report, noting clearly if any steps used simulated outputs due to AWS Academy IAM restrictions.
